# Operar con datos en Pandas

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones rápidas entre elementos, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.).
Pandas hereda gran parte de esta funcionalidad de NumPy, y las ufuncs son clave para ello.

Pandas incluye un par de giros útiles, sin embargo: para operaciones unarias como la negación y las funciones trigonométricas, estas ufuncs *preservarán las etiquetas de índice y columna* en la salida, y para operaciones binarias como la suma y la multiplicación, Pandas *alineará automáticamente los índices* al pasar los objetos a la ufunc.
Esto significa que mantener el contexto de los datos y combinar datos de diferentes fuentes -ambas tareas potencialmente propensas a errores con arrays NumPy sin procesar- se convierten en tareas esencialmente infalibles con Pandas.
Además veremos que hay operaciones bien definidas entre estructuras unidimensionales ``Series`` y estructuras bidimensionales ``DataFrame``.

## Ufuncs: Preservación de Índice

Como Pandas está diseñado para trabajar con NumPy, cualquier ufunc de NumPy funcionará con los objetos ``Series`` y ``DataFrame`` de Pandas.
Empecemos definiendo una ``Series`` y un ``DataFrame`` simples para demostrarlo:

In [38]:
import pandas as pd
import numpy as np

In [39]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4)) # Deveulve una lista de 4 elementos aleatorios entre 0 y 10
ser

0    6
1    3
2    7
3    4
dtype: int32

In [40]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), # Deveulve una lista de 3 filas y 4 columnas con elementos aleatorios entre 0 y 10
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


Si aplicamos una ufunc NumPy sobre cualquiera de estos objetos, el resultado será otro objeto Pandas *con los índices conservados:*.

In [41]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

O, para un cálculo un poco más complejo:

In [42]:
np.sin(df * np.pi / 4) # Dtaframe por pi entre cuatro y luego toto es se hace el coseno

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## UFuncs: Alineación de índices

Para operaciones binarias sobre dos objetos ``Series`` o ``DataFrame``, Pandas alineará los índices en el proceso de realización de la operación.
Esto es muy conveniente cuando se trabaja con datos incompletos, como veremos en algunos de los ejemplos que siguen.

### Alineación de índices en serie

Como ejemplo, supongamos que estamos combinando dos fuentes de datos diferentes, y encontramos sólo los tres primeros estados de EE.UU. por *área* y los tres primeros estados de EE.UU. por *población*:

In [43]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area') # tengo alaska y no NY
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population') # Tengo NY y no alaska 

Veamos qué ocurre cuando los dividimos para calcular la densidad de población:

In [44]:
population / area # Cuando falta un elemento numpy lo rellena con NaN

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

La matriz resultante contiene la *unión* de los índices de las dos matrices de entrada, que podría determinarse utilizando la aritmética de conjuntos estándar de Python sobre estos índices:

In [45]:
area.index.union(population.index) # Los une

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Cualquier elemento para el que uno u otro no tenga una entrada se marca con ``NaN``, o "Not a Number", que es como Pandas marca los datos que faltan.
Esta coincidencia de índices se implementa de esta forma para cualquiera de las expresiones aritméticas incorporadas en Python; cualquier valor que falte se rellena con NaN por defecto:

In [46]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [47]:
A

0    2
1    4
2    6
dtype: int64

Si el uso de valores NaN no es el comportamiento deseado, el valor de relleno puede modificarse utilizando métodos de objeto apropiados en lugar de los operadores.
Por ejemplo, llamar a ``A.add(B)`` es equivalente a llamar a ``A + B``, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento de ``A`` o ``B`` que pueda faltar:

In [48]:
A.add(B, fill_value=0) # A + B y dodne no hay nada coloca un 0 

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Alineación de índices en DataFrame

Un tipo similar de alineación tiene lugar para *tanto* columnas como índices cuando se realizan operaciones en ``DataFrame``:

In [49]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [50]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [51]:
A + B # Suma elemento a elemento donde se pueda

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


Observe que los índices se alinean correctamente independientemente de su orden en los dos objetos, y que los índices del resultado están ordenados.
Como en el caso de ``Series``, podemos utilizar el método aritmético del objeto asociado y pasarle cualquier ``fill_value`` que queramos utilizar en lugar de las entradas que falten.
Aquí rellenaremos con la media de todos los valores de ``A`` (calculada apilando primero las filas de ``A``):

In [52]:
A.stack()

0  A     1
   B    11
1  A     5
   B     1
dtype: int32

In [53]:
A.stack().mean() # La media de todo los valores juntos

np.float64(4.5)

In [54]:
A.mean() 

A    3.0
B    6.0
dtype: float64

In [55]:
A+B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


In [56]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


La siguiente tabla lista los operadores de Python y sus métodos equivalentes en los objetos de Pandas:

| Operador Python | Método(s) Panda                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operaciones entre DataFrame y Series

Cuando se realizan operaciones entre un ``DataFrame`` y una ``Series``, la alineación de índices y columnas se mantiene de forma similar.
Las operaciones entre un ``DataFrame`` y una ``Series`` son similares a las operaciones entre un array NumPy bidimensional y unidimensional.
Consideremos una operación común, donde encontramos la diferencia de un array bidimensional y una de sus filas:

In [57]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]], dtype=int32)

In [58]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]], dtype=int32)

Según las reglas de difusión de NumPy, la resta entre una matriz bidimensional y una de sus filas se aplica fila a fila.

En Pandas, la convención opera de forma similar por defecto:

In [62]:
df = pd.DataFrame(A, columns=list('QRST'))
#df - df.iloc[0]
df

,Q,R,S,T
0,3,8,2,4
1,2,6,4,8
2,6,1,3,8


Si, por el contrario, desea operar por columnas, puede utilizar los métodos de objeto mencionados anteriormente, especificando la palabra clave ``axis``:

In [60]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


Tambien es posible utilizar las `columns` o `index` para evitar confusion con el axis `0` o `1`

In [ ]:
df.subtract(df['R'],axis='index') # Sale todo 0 en la R

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


Tenga en cuenta que estas operaciones ``DataFrame``/``Series``, al igual que las operaciones comentadas anteriormente, alinearán automáticamente los índices entre los dos elementos:

In [ ]:
halfrow = df.iloc[0, ::2] # de 2 en dos desde el principio hasta el final (la mitad de la fila)
halfrow

Q    3
S    2
Name: 0, dtype: int32

In [65]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


Esta preservación y alineación de índices y columnas significa que las operaciones sobre datos en Pandas siempre mantendrán el contexto de los datos, lo que previene los tipos de errores tontos que podrían surgir al trabajar con datos heterogéneos y/o desalineados en arrays NumPy sin procesar.

<!--NAVIGATION-->
< [Indexación y selección de datos](2-Indexacion_y_seleccion_de_datos.ipynb) | [Tratamiento de los datos missing](4-Valores_missing.ipynb) >
